# Convert Text to IDs

## Content
1. [Import data](#import)
2. [extract data and add IDs](#extract)
3. [save to csv](#save-to-csv)

In [14]:
# imports
import pandas as pd
from db import get_database

<a id='import'></a>
## Import data

In [15]:
# get data from mongodb
dbname = get_database()
collection = dbname["joined"]

documents = collection.find()
df = pd.DataFrame(list(documents))

df.head(2)

,_id,display,occupation,tasks,technology_skills,tools_used,tools_technology,knowledge,skills,abilities,...,work_context,job_zone,education,interests,work_styles,work_values,related_occupations,additional_information,isco08,Name_de
0,6649fb988a508636be6bfa35,long,"{'code': '27-2011.00', 'title': 'Actors', 'tag...","{'task': [{'id': 7646, 'green': False, 'relate...",{'category': [{'related': 'https://services.on...,{'category': [{'related': 'https://services.on...,{'tools': {'category': [{'related': 'https://s...,"{'element': [{'id': '2.C.7.c', 'related': 'htt...","{'element': [{'id': '2.A.1.a', 'related': 'htt...","{'element': [{'id': '1.A.1.a.3', 'related': 'h...",...,"{'element': [{'id': '4.C.1.b.1.e', 'related': ...","{'value': 2, 'title': 'Job Zone Two: Some Prep...",{'level_required': {'category': [{'name': 'Les...,"{'element': [{'id': '1.B.1.c', 'related': 'htt...","{'element': [{'id': '1.C.3.a', 'related': 'htt...","{'element': [{'id': '1.B.2.d', 'related': 'htt...",{'occupation': [{'href': 'https://services.one...,{'source': [{'url': 'https://www.actorsequity....,2655,Schauspieler
1,6649fb988a508636be6bfa36,long,"{'code': '23-1021.00', 'title': 'Administrativ...","{'task': [{'id': 7627, 'green': False, 'relate...",{'category': [{'related': 'https://services.on...,{'category': [{'related': 'https://services.on...,{'tools': {'category': [{'related': 'https://s...,"{'element': [{'id': '2.C.8.b', 'related': 'htt...","{'element': [{'id': '2.A.1.b', 'related': 'htt...","{'element': [{'id': '1.A.1.b.5', 'related': 'h...",...,"{'element': [{'id': '4.C.2.a.1.a', 'related': ...","{'value': 5, 'title': 'Job Zone Five: Extensiv...",{'level_required': {'category': [{'name': 'Doc...,"{'element': [{'id': '1.B.1.f', 'related': 'htt...","{'element': [{'id': '1.C.5.c', 'related': 'htt...","{'element': [{'id': '1.B.2.a', 'related': 'htt...",{'occupation': [{'href': 'https://services.one...,{'source': [{'url': 'https://www.americanbar.o...,2612,Richter


In [16]:
# create dfs
skills = pd.DataFrame()
abilities = pd.DataFrame()

<a id='extract'></a>
## Extract data and add IDs

In [17]:
# extract all skills from the onet data
skills = df['skills'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
skills_df = pd.json_normalize(skills)

# Drop duplicates based on 'id'
skills_df = skills_df.drop_duplicates(subset='id')

# Reset the index
skills_df = skills_df.reset_index(drop=True)

# drop cols
skills_df = skills_df.drop(columns=["related", "score.scale", "score.important", "score.value"])

# add numeric id col
skills_df['skill_id'] = range(1, len(skills_df) + 1)

skills_df.head(3)

,id,name,description,skill_id
0,2.A.1.a,Reading Comprehension,Understanding written sentences and paragraphs...,1
1,2.A.1.d,Speaking,Talking to others to convey information effect...,2
2,2.A.1.b,Active Listening,Giving full attention to what other people are...,3


In [18]:
# extract all abilities from the onet data
abilities = df['abilities'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
abilities_df = pd.json_normalize(abilities)

# Drop duplicates based on 'id'
abilities_df = abilities_df.drop_duplicates(subset='id')

# Reset the index
abilities_df = abilities_df.reset_index(drop=True)

# drop cols
abilities_df = abilities_df.drop(columns=["related", "score.scale", "score.important" ,"score.value"])

# add numeric id col
abilities_df['ability_id'] = range(1, len(abilities_df) + 1)

abilities_df.head(3)

,id,name,description,ability_id
0,1.A.1.a.3,Oral Expression,The ability to communicate information and ide...,1
1,1.A.1.a.1,Oral Comprehension,The ability to listen to and understand inform...,2
2,1.A.1.d.1,Memorization,The ability to remember information such as wo...,3


<a id='save-to-csv'></a>
## Save to csv

In [19]:

abilities_df.to_csv("files/onet_abilities.csv", index=False)
skills_df.to_csv("files/onet_skills.csv", index=False)